In [1]:
!pip install --user gurobipy
import sys
print(sys.executable)





/opt/anaconda3/bin/python


In [17]:
import pandas as pd
from gurobipy import Model, GRB
import math

# Load the data from the Excel file
excel_file = "Scheduling Project Pilot (5).xlsx"
df = pd.read_excel(excel_file, sheet_name='Assignments_')
df = df.rename(columns={df.columns[0]: 'Course'})
df = df.rename(columns={df.columns[1]: 'Instructor'})
# Further filter the DataFrame to include only courses after "CS113" and exclude "CS115"
df = df[df['Course'].str.startswith('CS')]
df['Course_Number'] = df['Course'].str.extract(r'CS(\d+)').astype(int)


new_excel_file = "Faculty and Instructors.xlsx"
new_df = pd.read_excel(new_excel_file,sheet_name='2024-09-Fall')
#print(new_df)

# Assuming the new_df also has an 'Instructor' column with professor names,
# Perform a merge on the 'Instructor' column
df = pd.merge(df, new_df, on='Instructor', how='left')
print(df)

# Drop the helper column used for filtering
#df = df.drop(columns=['Course_Number'])

# Define the time slots and days (including Saturday, but minimizing its usage)
time_slots = [
    "8:30-10:00 AM",
    "10:00-11:30 AM",
    "11:30-1:00 PM",
    "1:00-2:30 PM",
    "2:30-4:00 PM",
    "4:00-5:30 PM",
    "6:00-7:30 PM",
    "7:30-9:00 PM"
]

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

# Load the balance percentages from the table in the image
slot_percentages = {
    ("Monday", "8:30-10:00 AM"): 0.20,
    ("Tuesday", "8:30-10:00 AM"): 0.20,
    ("Wednesday", "8:30-10:00 AM"): 0.25,
    ("Thursday", "8:30-10:00 AM"): 0.20,
    ("Friday", "8:30-10:00 AM"): 0.20,
    
    
    ("Monday", "10:00-11:30 AM"): 0.20,
    ("Tuesday", "10:00-11:30 AM"): 0.20,
    ("Wednesday", "10:00-11:30 AM"): 0.25,
    ("Thursday", "10:00-11:30 AM"): 0.20,
    ("Friday", "10:00-11:30 AM"): 0.20,
    
    
    ("Monday", "11:30-1:00 PM"): 0.20,
    ("Tuesday", "11:30-1:00 PM"): 0.20,
    ("Wednesday", "11:30-1:00 PM"): 0.25,
    ("Thursday", "11:30-1:00 PM"): 0.20,
    ("Friday", "11:30-1:00 PM"): 0.00,
    
    
    ("Monday", "1:00-2:30 PM"): 0.20,
    ("Tuesday", "1:00-2:30 PM"): 0.20,
    ("Wednesday", "1:00-2:30 PM"): 0.25,
    ("Thursday", "1:00-2:30 PM"): 0.20,
    ("Friday", "1:00-2:30 PM"): 0.20,
    
    
    ("Monday", "2:30-4:00 PM"): 0.20,
    ("Tuesday", "2:30-4:00 PM"): 0.20,
    ("Wednesday", "2:30-4:00 PM"): 0.00,  
    ("Thursday", "2:30-4:00 PM"): 0.20,
    ("Friday", "2:30-4:00 PM"): 0.20,
    
    ("Monday", "4:00-5:30 PM"): 0.20,
    ("Tuesday", "4:00-5:30 PM"): 0.20,
    ("Wednesday", "4:00-5:30 PM"): 0.00,  
    ("Thursday", "4:00-5:30 PM"): 0.20,
    ("Friday", "4:00-5:30 PM"): 0.20,
    
    # Evening slots
    ("Monday", "6:00-7:30 PM"): 0.20,
    ("Tuesday", "6:00-7:30 PM"): 0.20,
    ("Wednesday", "6:00-7:30 PM"): 0.20,
    ("Thursday", "6:00-7:30 PM"): 0.20,
    ("Friday", "6:00-7:30 PM"): 0.20,
    
    ("Monday", "7:30-9:00 PM"): 0.20,
    ("Tuesday", "7:30-9:00 PM"): 0.20,
    ("Wednesday", "7:30-9:00 PM"): 0.20,
    ("Thursday", "7:30-9:00 PM"): 0.20,
    ("Friday", "7:30-9:00 PM"): 0.20
}
variables = {}
model = Model("Scheduling")


# Add slack variables and update the objective to minimize slack
# Add slack variables and update the objective to minimize slack
slack_vars = {}
total_slack = 0  # Initialize total slack

for (day, slot), percentage in slot_percentages.items():
    if percentage > 0:  # Only define slack variables for non-zero percentages
        slack_var_name = f"Slack_{day}_{slot}"
        slack_vars[slack_var_name] = model.addVar(vtype=GRB.CONTINUOUS, name=slack_var_name)
        total_slack += slack_vars[slack_var_name]

# Update the constraint for the maximum allowed parts



# Define the binary variables


for _, row in df.iterrows():
    course = row['Course']
    instructor = row['Instructor']
    num_sections = int(row['Sum of D'])

    if num_sections == 0:
        continue

    for section_id in range(1, num_sections + 1):
        for part in [1, 2]:  # 2x the number of sections
            for day in days:
                for slot in time_slots:
                    var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                    variables[var_name] = model.addVar(vtype=GRB.BINARY, name=var_name)
# Calculate total number of section parts (2N)
total_section_parts = 2 * sum(int(row['Sum of D']) for _, row in df.iterrows())

for (day, slot), percentage in slot_percentages.items():
    slot_vars = [variables[var_name] for var_name in variables if var_name.split('_')[5] == day and var_name.split('_')[6] == slot]
    
    max_section_parts_slot = math.ceil((percentage / 6) * total_section_parts)
    
    if percentage > 0:  # Only add slack variables if the percentage is non-zero
        model.addConstr(
            sum(slot_vars) <= max_section_parts_slot + slack_vars[f"Slack_{day}_{slot}"], 
            name=f"balance_slot_with_slack_{day}_{slot}"
        )
    else:
        model.addConstr(
            sum(slot_vars) <= max_section_parts_slot, 
            name=f"balance_slot_no_slack_{day}_{slot}"
        )
    
    print(f"Constraint added for {day} {slot}: max {max_section_parts_slot} section parts")


    
evening_percentage = 0.20  # Evening slots defined as 6:00-7:30 PM and 7:30-9:00 PM

evening_slots = ["6:00-7:30 PM", "7:30-9:00 PM"]
for slot in evening_slots:
    evening_vars = [variables[var_name] for var_name in variables if var_name.split('_')[6] == slot]
    max_section_parts_evening = evening_percentage / 6 * total_section_parts
    model.addConstr(sum(evening_vars) <= max_section_parts_evening, name=f"balance_evening_{slot}")


# Set the objective to maximize the number of graduate courses scheduled on consecutive slots



# Add the constraints with numbering and spacing
constraint_counter = 0

for _, row in df.iterrows():
    course = row['Course']
    instructor = row['Instructor']
    num_sections = int(row['Sum of D'])

    if num_sections == 0:
        continue

    for section_id in range(1, num_sections + 1):
        for part in range(1, 3):
            section_vars = []
            for day in days:
                for slot in time_slots:
                    var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                    if var_name in variables:
                        var = variables[var_name]
                        section_vars.append(var)

            if section_vars:
                constraint_name = f"unique_slot_{course}_{instructor}_{section_id}_{part}"
                model.addConstr(sum(section_vars) == 1, name=constraint_name)
                constraint_counter += 1
# Ensure an instructor is not scheduled for more than one section at the same time slot
for instructor in df['Instructor'].unique():
    for day in days:
        for slot in time_slots:
            instructor_vars = []
            for _, row in df[df['Instructor'] == instructor].iterrows():
                course = row['Course']
                num_sections = int(row['Sum of D'])

                for section_id in range(1, num_sections + 1):
                    for part in range(1, 3):
                        var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                        if var_name in variables:
                            var = variables[var_name]
                            instructor_vars.append(var)

            if instructor_vars:
                constraint_name = f"one_section_per_slot_{instructor}_{day}_{slot}"
                model.addConstr(sum(instructor_vars) <= 1, name=constraint_name)

                
# Define valid start times for graduate courses
valid_start_times = ["8:30-10:00 AM", "11:30-1:00 PM", "2:30-4:00 PM", "6:00-7:30 PM"]

for course in df['Course'].unique():
    for instructor in df['Instructor'].unique():
        course_instructor_rows = df[(df['Course'] == course) & (df['Instructor'] == instructor)]
        if course_instructor_rows.empty:
            continue

        course_number = int(course_instructor_rows['Course_Number'].iloc[0])

        for section_id in range(1, course_instructor_rows.iloc[0]['Sum of D'] + 1):
            for slot in time_slots:

                # Constraints for courses with number < 600 (undergraduate)
                if course_number < 600:
                    var_part1_monday = f"X_{course}_{instructor}_{section_id}_1_Monday_{slot}"
                    var_part1_tuesday = f"X_{course}_{instructor}_{section_id}_1_Tuesday_{slot}"
                    var_part1_wednesday = f"X_{course}_{instructor}_{section_id}_1_Wednesday_{slot}"
                    var_part1_thursday = f"X_{course}_{instructor}_{section_id}_1_Thursday_{slot}"
                    var_part1_friday = f"X_{course}_{instructor}_{section_id}_1_Friday_{slot}"

                    # Part 1 on Monday: Part 2 on Wednesday or Thursday, same time slot
                    if var_part1_monday in variables:
                        var_part2_wednesday = f"X_{course}_{instructor}_{section_id}_2_Wednesday_{slot}"
                        var_part2_thursday = f"X_{course}_{instructor}_{section_id}_2_Thursday_{slot}"
                        model.addConstr(
                            variables[var_part1_monday] <= variables[var_part2_wednesday] + variables[var_part2_thursday],
                            name=f"part1_monday_part2_wed_thurs_{course}_{instructor}_{section_id}_{slot}"
                        )

                    # Part 1 on Tuesday: Part 2 on Thursday or Friday, same time slot
                    if var_part1_tuesday in variables:
                        var_part2_thursday = f"X_{course}_{instructor}_{section_id}_2_Thursday_{slot}"
                        var_part2_friday = f"X_{course}_{instructor}_{section_id}_2_Friday_{slot}"
                        model.addConstr(
                            variables[var_part1_tuesday] <= variables[var_part2_thursday] + variables[var_part2_friday],
                            name=f"part1_tuesday_part2_thurs_fri_{course}_{instructor}_{section_id}_{slot}"
                        )

                    # Part 1 on Wednesday: Part 2 on Friday, same time slot
                    if var_part1_wednesday in variables:
                        var_part2_friday = f"X_{course}_{instructor}_{section_id}_2_Friday_{slot}"
                        model.addConstr(
                            variables[var_part1_wednesday] <= variables[var_part2_friday],
                            name=f"part1_wednesday_part2_friday_{course}_{instructor}_{section_id}_{slot}"
                        )

                    # Prevent scheduling Part 1 on Thursday or Friday unless both parts are on the same day
                    if var_part1_thursday in variables:
                        model.addConstr(variables[var_part1_thursday] == 0, name=f"no_part1_thursday_{course}_{instructor}_{section_id}_{slot}")
                    if var_part1_friday in variables:
                        model.addConstr(variables[var_part1_friday] == 0, name=f"no_part1_friday_{course}_{instructor}_{section_id}_{slot}")

                

                   # For courses with number >= 600, schedule parts on consecutive time slots,
# with the first time slot starting at one of the valid start times.
                if course_number >= 600:
                    consecutive_schedule_vars = []  # Track consecutive scheduling variables for the section
                    
                    for day in days:
                        for idx, slot in enumerate(time_slots[:-1]):  # Exclude last slot for consecutive check
                            var_part1 = f"X_{course}_{instructor}_{section_id}_1_{day}_{slot}"
                            var_part2 = f"X_{course}_{instructor}_{section_id}_2_{day}_{time_slots[idx + 1]}"
                
                            # Only apply this constraint if the first time slot is one of the valid start times
                            if slot in valid_start_times and var_part1 in variables and var_part2 in variables:
                                # Create a binary variable to indicate whether this pair of time slots is used
                                var_consecutive = model.addVar(vtype=GRB.BINARY, name=f"Consecutive_{course}_{instructor}_{section_id}_{day}_{slot}")
                                consecutive_schedule_vars.append(var_consecutive)
                
                                # Ensure that if this consecutive pair is selected, both parts are scheduled
                                model.addConstr(
                                    variables[var_part1] == var_consecutive,
                                    name=f"link_part1_{course}_{instructor}_{section_id}_{day}_{slot}"
                                )
                                model.addConstr(
                                    variables[var_part2] == var_consecutive,
                                    name=f"link_part2_{course}_{instructor}_{section_id}_{day}_{slot}"
                                )
                
                    # Ensure that exactly one valid consecutive pair is chosen for the section
                    model.addConstr(
                        sum(consecutive_schedule_vars) == 1,  # Only one consecutive pair should be selected
                        name=f"one_consecutive_pair_{course}_{instructor}_{section_id}"
                    )

# Define the specific time slot and day for the constraint
restricted_day = "Monday"
restricted_time_slot = "4:00-5:30 PM"

# Add the constraint: Only courses with course number > 199 and "Sum of C" < 35 can be scheduled on Monday from 4:00 to 5:30
for _, row in df.iterrows():
    course = row['Course']
    instructor = row['Instructor']
    course_number = int(row['Course_Number'])
    sum_of_c = int(row['Sum of C'])

    # Check if the course meets the condition for being scheduled in this restricted slot
    if course_number > 199 and sum_of_c < 35:
        # Loop over sections and parts to ensure the variables for this course are allowed to be scheduled
        for section_id in range(1, int(row['Sum of D']) + 1):
            for part in [1, 2]:
                var_name = f"X_{course}_{instructor}_{section_id}_{part}_{restricted_day}_{restricted_time_slot}"
                if var_name in variables:
                    # No constraint needed, the course meets the condition
                    continue
    else:
        # If the course does not meet the conditions, add a constraint to prevent it from being scheduled in the restricted slot
        for section_id in range(1, int(row['Sum of D']) + 1):
            for part in [1, 2]:
                var_name = f"X_{course}_{instructor}_{section_id}_{part}_{restricted_day}_{restricted_time_slot}"
                if var_name in variables:
                    model.addConstr(variables[var_name] == 0, name=f"restricted_slot_{course}_{instructor}_{section_id}_{part}_{restricted_day}_{restricted_time_slot}")

# Set slack variables to zero manually
#for slack_var_name, slack_var in slack_vars.items():
#    model.addConstr(slack_var == 0, name=f"set_{slack_var_name}_to_zero")
# Define the course blocks
course_blocks = [
    ['CS114', 'IS210', 'CS450', 'CS337'],
    ['CS241', 'CS280', 'IS350'],
    ['CS288', 'CS332', 'CS301', 'CS356'],
    ['CS341', 'CS350', 'CS351', 'CS331', 'CS375'],
    ['CS435', 'CS490', 'CS485', 'CS370', 'CS375'],
    ['CS485', 'CS491', 'CS450', 'CS482'],
    ['CS610', 'CS630', 'CS631', 'CS656', 'DS675', 'CS675', 'CS670'],  # Block-grad-core
    ['DS677', 'DS669', 'DS650', 'CS670', 'CS610', 'CS665', 'CS667','CS732',	'DS680'],  # Block-grad-DS+Alg
    ['CS608', 'CS645', 'CS646', 'CS647', 'CS648', 'CS678', 'CS696']   # Block-grad-cyber
]

# Add constraint to ensure no different parts of courses in the same block are scheduled in the same day and time slot
for block in course_blocks:
    for course1 in block:
        for course2 in block:
            if course1 != course2:
                for instructor1 in df[df['Course'] == course1]['Instructor'].unique():
                    for instructor2 in df[df['Course'] == course2]['Instructor'].unique():
                        for day in days:
                            for slot in time_slots:
                                var_course1_part1 = f"X_{course1}_{instructor1}_1_{day}_{slot}"
                                var_course1_part2 = f"X_{course1}_{instructor1}_2_{day}_{slot}"
                                var_course2_part1 = f"X_{course2}_{instructor2}_1_{day}_{slot}"
                                var_course2_part2 = f"X_{course2}_{instructor2}_2_{day}_{slot}"

                                # Add constraints to prevent parts of different courses from being scheduled in the same day and slot
                                if var_course1_part1 in variables and var_course2_part1 in variables:
                                    model.addConstr(
                                        variables[var_course1_part1] + variables[var_course2_part1] <= 1,
                                        name=f"no_same_day_slot_{course1}_{instructor1}_{course2}_{instructor2}_{day}_{slot}_part1"
                                    )
                                if var_course1_part2 in variables and var_course2_part2 in variables:
                                    model.addConstr(
                                        variables[var_course1_part2] + variables[var_course2_part2] <= 1,
                                        name=f"no_same_day_slot_{course1}_{instructor1}_{course2}_{instructor2}_{day}_{slot}_part2"
                                    )



# This dictionary maps time slot abbreviations to their respective slot index
time_slot_mapping = {
    'M': 'Monday', 
    'T': 'Tuesday', 
    'W': 'Wednesday', 
    'R': 'Thursday', 
    'F': 'Friday',
    'S': 'Saturday'  # Define 'S' for completeness, but we'll ignore it
}

# Time slot indexes
time_slot_index = {
    '1': "8:30-10:00 AM",
    '2': "10:00-11:30 AM",
    '3': "11:30-1:00 PM",
    '4': "1:00-2:30 PM",
    '5': "2:30-4:00 PM",
    '6': "4:00-5:30 PM",
    '7': "6:00-7:30 PM",
    '8': "7:30-9:00 PM"
}

# Sample data format:
# instructor_constraints = {
#     "ahh2@njit.edu": ["M2", "R2", "M4", "T4", "M5", "T5", "T7", "T8"],
#     "alexg@njit.edu": ["M5", "R3"]
# }

# Add the constraints for instructors who have "Health" or "Religion" type
df_constraints = pd.read_excel(excel_file, sheet_name='Constraints & Preferences')
for _, row in df_constraints.iterrows():
    instructor_info = row['Instructor UCID: Type']
    slots = row['Slots']

    # Parse the instructor UCID and type
    email, constraint_type = instructor_info.split(": ")
    
    # We only care about the instructors with "Health" or "Religion" type
    if constraint_type.strip() in ["Health", "Religion"]:
        # Parse the blocked time slots for this instructor
        blocked_slots = slots.split("|")[1:-1]  # Remove empty elements from split
        
        for slot_code in blocked_slots:
            # Extract the day and time slot
            day_abbrev = slot_code[0]  # M, T, W, R, F
            time_slot_num = slot_code[1]  # 1-8
            
            day_full = time_slot_mapping[day_abbrev]
            time_slot_full = time_slot_index[time_slot_num]
            
            # Add a constraint to block this time slot for all parts of the instructor's courses
            for course in df[df['Instructor'] == email]['Course']:
                num_sections = int(df[df['Course'] == course]['Sum of D'].iloc[0])
                
                for section_id in range(1, num_sections + 1):
                    for part in [1, 2]:  # 2 parts per section
                        var_name = f"X_{course}_{email}_1_{day_full}_{time_slot_full}"
                        
                        # Ensure the variable exists in the model
                        if var_name in variables:
                            model.addConstr(
                                variables[var_name] == 0, 
                                name=f"block_slot_{email}_{day_full}_{time_slot_full}"
                            )




model.setObjective(total_slack, GRB.MINIMIZE)
                       
# Update the model
model.update()

# Solve the model
model.optimize()

if model.Status == GRB.INFEASIBLE:
    # If the model is infeasible, compute the IIS and write the ILP model to a text file
    print("The model is infeasible; computing IIS")
    model.computeIIS()

    with open("model_with_constraints.txt", "w") as f:
        f.write("Infeasible Model with Numbered Constraints\n\n")
        for i, constr in enumerate(model.getConstrs(), 1):
            if constr.IISConstr:
                f.write(f"Constraint {i}: {constr.ConstrName}\n")
                f.write(f"{model.getRow(constr)} = {constr.RHS}\n\n")

    print("IIS written to model_with_constraints.txt")

if model.Status == GRB.OPTIMAL:
    # If the model is feasible, extract and write the schedule to a text file
    schedule = []
    slack_values = []

    for var in model.getVars():
        if var.varName.startswith('Slack_'):
            # Store slack variables and their values
            slack_values.append((var.varName, var.x))
        elif var.x > 0.5:  # If the variable is selected in the optimal solution
            name = var.varName
            try:
                _, course, instructor, section_id, part, day, slot = name.split('_')
                # Find the corresponding email for the instructor in the DataFrame
                email = df.loc[df['Instructor'] == instructor, 'Email'].values[0]  # Assuming 'Email' column contains the emails
                schedule.append((course, instructor, email, section_id, part, day, slot))
            except ValueError:
                # Handle any variables that may not follow the expected naming pattern
                pass

    # Sort the schedule lexicographically
    schedule.sort()

    # Write the lexicographically sorted schedule and slack values to a file
    with open("final_schedule_with_slack.txt", "w") as f:
        f.write("Course Schedule (Lexicographically Sorted):\n\n")
        for entry in schedule:
            course, instructor, email, section_id, part, day, slot = entry
            f.write(f"Course: {course}, Instructor: {instructor}, Email: {email}, Section: {section_id}, Part: {part}, Day: {day}, Slot: {slot}\n")
        
        f.write("\nSlack Variable Values:\n\n")
        for slack_var, value in slack_values:
            f.write(f"{slack_var}: {value}\n")

    print("Final schedule with slack values written to final_schedule_with_slack.txt")

    # Sort the schedule based on the instructor's name
    schedule_sorted_by_instructor = sorted(schedule, key=lambda x: x[1])  # Sort by the second element (instructor)

    # Write the instructor-sorted schedule and slack values to a separate file, grouped by instructor
    with open("final_schedule_sorted_by_instructor.txt", "w") as f:
        f.write("Course Schedule (Sorted by Instructor):\n\n")
        
        current_instructor = None
        for entry in schedule_sorted_by_instructor:
            course, instructor, email, section_id, part, day, slot = entry
            
            # If we encounter a new instructor, print their email and name first
            if instructor != current_instructor:
                if current_instructor is not None:
                    f.write("\n")  # Separate different instructors' sections
                
                f.write(f"Instructor: {instructor}, Email: {email}\n")
                current_instructor = instructor
            
            # Write the course details for the current instructor
            f.write(f"\tCourse: {course}, Section: {section_id}, Part: {part}, Day: {day}, Slot: {slot}\n")
        
        f.write("\nSlack Variable Values:\n\n")
        for slack_var, value in slack_values:
            f.write(f"{slack_var}: {value}\n")

    print("Final schedule sorted by instructor written to final_schedule_sorted_by_instructor.txt")






   Course          Instructor  Sum of C  Sum of D Unnamed: 4  Course_Number  \
0   CS114   Kapleau, Jonathan        43         2     Newark            114   
1   CS114  Zaidenberg, Ayelet        80         1     Newark            114   
2   CS115  Qerimaj, Jertishta       100         3     Newark            115   
3   CS115     Soltis, Jolanta        30         2     Newark            115   
4   CS116             Wu, Jun        30         2     Newark            116   
..    ...                 ...       ...       ...        ...            ...   
66  CS698        Wang, Lijing        30         1     Newark            698   
67  CS732            Wei, Zhi        43         1     Newark            732   
68  CS785   Basu Roy, Senjuti        20         1     Newark            785   
69  CS785     Neamtiu, Iulian        40         1     Newark            785   
70  CS785    Sharma, Shantanu        20         1     Newark            785   

                Email                 Rank Departme